In [1]:
import numpy as np
import pandas as pd

In [2]:
# we can use multi-indexing objects to represent higher-dimensional data in a lower-dimensional form
index = [
    ("California", 2010),
    ("California", 2020),
    ("New York", 2010),
    ("New York", 2020),
    ("Texas", 2010),
    ("Texas", 2020),
]
populations = [37253956, 39538223, 19378102, 20201249, 25145561, 29145505]
populations = pd.Series(populations, index=index)
index = pd.MultiIndex.from_tuples(index)
populations

(California, 2010)    37253956
(California, 2020)    39538223
(New York, 2010)      19378102
(New York, 2020)      20201249
(Texas, 2010)         25145561
(Texas, 2020)         29145505
dtype: int64

In [3]:
populations = populations.reindex(index)
populations

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [4]:
populations[:, 2020]

California    39538223
New York      20201249
Texas         29145505
dtype: int64

In [5]:
# multiindex as extra dimension
# the unstack() method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame
populations_df = populations.unstack()
populations_df

,2010,2020
California,37253956,39538223
New York,19378102,20201249
Texas,25145561,29145505


In [6]:
# the stack() method provides the opposite operation
populations_df.stack()

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [7]:
# just as we were able to use multi-indexing to maipulate two-dimensional data within a one-dimensional Series object, we can also use it to manipulate data in a three-dimensional DataFrame each extra level in a multi-index represents an extra dimension of data. taking advantage of this property gives us much more flexibility in the types of data we can represent. we can add another column of demographic data for each state at each year
populations_df = pd.DataFrame(
    {
        "total": populations,
        "under18": [9284094, 8898092, 4318033, 4181528, 6879014, 7432474],
    }
)
populations_df

total  under18
California 2010  37253956  9284094
           2020  39538223  8898092
New York   2010  19378102  4318033
           2020  20201249  4181528
Texas      2010  25145561  6879014
           2020  29145505  7432474

In [8]:
# in addition to all the ufuncs and other functionality discussed, work with hierarchical indices as well
# we can compuute the fraction of people under 18 by year, given the above data
f_u18 = populations_df["under18"] / populations_df["total"]
f_u18.unstack()
# this allows us to easily and quickly manipulate and explore even high-dimensional data

,2010,2020
California,0.249211,0.225050
New York,0.222831,0.206994
Texas,0.273568,0.255013


In [9]:
# methods of multiindex creation
# the most straightforward way to create a multiply indexed Series or DataFrame is to simply pass a list of two or more index arrays to the constructor
df = pd.DataFrame(
    np.random.rand(4, 2),
    index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
    columns=["data1", "data2"],
)
df

data1     data2
a 1  0.132802  0.223230
  2  0.683607  0.155692
b 1  0.175699  0.272593
  2  0.228393  0.887819

In [10]:
# similarly if you pass a dictionary with appropriate tuples as keys, pandas will autonatically recognize this and use a MultiIndex by default
data = {
    ("California", 2010): 37253956,
    ("California", 2020): 39538223,
    ("New York", 2010): 19378102,
    ("New York", 2020): 20201249,
    ("Texas", 2010): 25145561,
    ("Texas", 2020): 29145505,
}
pd.Series(data)

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [11]:
# for more flexibility in how the index is constructed we can instead use the class method constructors available in the pd.MultiIndex class
# we can construct a multi-index from a simple list of arrays, giving the index values within each level
pd.MultiIndex.from_arrays([["a", "a", "b", "b"], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [12]:
# or you can construct it from a list of tuples, giving the multiple index values of each point
pd.MultiIndex.from_tuples([("a", 1), ("a", 2), ("b", 1), ("b", 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [13]:
# can even construct it from a cartesian product of single indices
pd.MultiIndex.from_product([["a", "b"], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [14]:
# we can even use internal encoding by passing levels and codes
pd.MultiIndex(levels=[["a", "b"], [1, 2]], codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
# sometimes it is convienient to name the levels of the multi-index. we can accomplish this by passing the names argument to any of the constructor methods, or by setting the names attribute of the index after the fact
populations.index.names = ["state", "year"]
populations

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [16]:
# in a dataframe the rows and columns are completely symmetirc, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=["year", "visit"])
columns = pd.MultiIndex.from_product(
    [["Bob", "Guido", "Sue"], ["HR", "Temp"]], names=["subject", "type"]
)
# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the dataframe
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data
# this is fundamentally four dimensional data where the dimensions are the subject the measurement type the year and the visit number. with this in place we can index the top level column by the person's name and get a full dataframe containing just that person's information

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      24.0  37.7  31.0  35.8  35.0  36.1
     2      29.0  35.7  35.0  35.3  31.0  35.8
2014 1      34.0  40.7  51.0  37.2  32.0  37.4
     2      39.0  37.6  45.0  37.7  34.0  36.5

In [17]:
health_data["Guido"]

type          HR  Temp
year visit            
2013 1      31.0  35.8
     2      35.0  35.3
2014 1      51.0  37.2
     2      45.0  37.7

In [18]:
# indexing and slicing a multiindex is designed to be intuitive
# multiply indexed series
populations

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [19]:
# we can access single elements by indexing with multiple terms
populations["California", 2010]

37253956

In [20]:
# the multiindex also supports partial indexing or indexing just one of the levels in the index. the result is another series with lower dimensional index
populations["California"]

year
2010    37253956
2020    39538223
dtype: int64

In [21]:
# partial slicing is available as well as long as the multiindex is sorted
populations.loc["California":"New York"]

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
dtype: int64

In [22]:
# with sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index
populations[:, 2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [23]:
# other types of indexing and selection work as well. for example selection based on boolean masks
populations[populations > 22000000]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

In [24]:
# selection based on fancy indexing also works
populations[["California", "Texas"]]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

In [25]:
# a multiply indexed dataframe behaves in a simalar manner.
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      24.0  37.7  31.0  35.8  35.0  36.1
     2      29.0  35.7  35.0  35.3  31.0  35.8
2014 1      34.0  40.7  51.0  37.2  32.0  37.4
     2      39.0  37.6  45.0  37.7  34.0  36.5

In [26]:
# columns are primary in a dataframe and the syntax used for multiply indexed series applies to the columns
# we can recover guido's heart rate data with a simple operation
health_data["Guido", "HR"]

year  visit
2013  1        31.0
      2        35.0
2014  1        51.0
      2        45.0
Name: (Guido, HR), dtype: float64

In [27]:
# also as with the single-index case we can use the loc, iloc, and ix indexers
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      24.0  37.7
     2      29.0  35.7

In [28]:
# these provide an array like view of the underlying two-dimensional data, but each individual index in loc or iloc can be passed a tuple of multiple indices for example
health_data.loc[:, ("Bob", "HR")]

year  visit
2013  1        24.0
      2        29.0
2014  1        34.0
      2        39.0
Name: (Bob, HR), dtype: float64

In [29]:
# working with slices within these index tuples is not convienient trying to create a slice within a tuple will lead to a syntax error
# health_data.loc[(:, 1), (:, 'HR')] # this will not work

In [30]:
# you can get around this by building the desired slice explicitly using python's built-in slice() function
# a better way in this context is to use an IndexSlice object which pandas provides for precisely this situation
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, "HR"]]
# there are many ways to interact with data in multiply indexed series and dataframes

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,24.0,31.0,35.0
2014,1,34.0,51.0,32.0


In [31]:
# rearranging multi-indices
# many of the MultiIndex slicing operations will fail if the index is not sorted.
# we create some simple multiply indexed data where the indices are not lexographically sorted
index = pd.MultiIndex.from_product([["a", "c", "b"], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ["char", "int"]
data

char  int
a     1      0.229033
      2      0.323188
c     1      0.780073
      2      0.233293
b     1      0.481781
      2      0.854125
dtype: float64

In [32]:
# if we try to take a partial slice of this index it will result in an error
try:
    data["a":"b"]
except KeyError as e:
    print("KeyError", e)

KeyError 'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [33]:
# this is the result of the multi-index not being sorted. for various reasons partial slices and other similar operations require the levels in the multi-index to be in sorted order. pandas provides a number of convenience routines to perform this type of sorting; examples are the sort_index() and sortlevel() methods of the dataframe. we'll use the simplest, sort_index(), here
data = data.sort_index()
data

char  int
a     1      0.229033
      2      0.323188
b     1      0.481781
      2      0.854125
c     1      0.780073
      2      0.233293
dtype: float64

In [34]:
# with the index sorted in this way, partial slicing will work as expected
data["a":"b"]

char  int
a     1      0.229033
      2      0.323188
b     1      0.481781
      2      0.854125
dtype: float64

In [35]:
# stacking and unstacking indices
# as we saw briefly before it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use
populations.unstack(level=0)

state,California,New York,Texas
year,,,
2010,37253956,19378102,25145561
2020,39538223,20201249,29145505


In [36]:
populations.unstack(level=1)

year,2010,2020
state,,
California,37253956,39538223
New York,19378102,20201249
Texas,25145561,29145505


In [37]:
populations.unstack().stack()

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

In [38]:
# index setting and resetting
# another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the reset_index method. Calling this on the population dictionary woill result ina  dataframe with state and year columns holding the information that was formerly in the index. for clarity we can optionally specify the name of the data for the column representation.
population_flat = populations.reset_index(name="populatiom")
population_flat

,state,year,populatiom
0,California,2010,37253956
1,California,2020,39538223
2,New York,2010,19378102
3,New York,2020,20201249
4,Texas,2010,25145561
5,Texas,2020,29145505


In [39]:
# a common pattern is to build a multi-index from the column values; this can be done with the set_index method of the dataframe, which returns a multiply indexed dataframe
population_flat.set_index(["state", "year"])

populatiom
state      year            
California 2010    37253956
           2020    39538223
New York   2010    19378102
           2020    20201249
Texas      2010    25145561
           2020    29145505

Combining datasets: concat and append

In [40]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)


# example dataframe
make_df("ABC", range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [41]:
# in addition we'll create a quick class that allows us to display multiple DataFrames side by side. the code makes use of the special _repr_html_ method, which ipython uses to implement its rich object display
class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return "\n".join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return "\n\n".join(a + "\n" + repr(eval(a)) for a in self.args)

In [42]:
# concatenation of numpy arrays
# can combine multiple numpy arrays into one
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [43]:
# the first argument is a list or tuple of arrays to concatenate. Additionally, in the case of multidimensional arrays, it takes an axis keyword that allows you to specify the axis along which the result will be concatenated
x = [[1, 2], [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

Simple concatenation with pd.concat

In [44]:
# the pd.concat function provides a similar syntax to np.concatenate but contains a number of options
# signature in pandas
"""
pd.concat(objs, axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False,
          sort=False, copy=True)
"""
ser1 = pd.Series(["A", "B", "C"], index=[1, 2, 3])
ser2 = pd.Series(["D", "E", "F"], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [45]:
# it also works to concatenate higher dimensional objects such as dataframes
df1 = make_df("AB", [1, 2])
df2 = make_df("AB", [3, 4])
display("df1", "df2", "pd.concat([df1, df2])")

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [46]:
# its default behavior is to concatenate row-wise within the dataframe (i.e., axis=0)
# like np.concatenate, pd.concat allows specification of an axis along which concatenation will take place.
df3 = make_df("AB", [0, 1])
df4 = make_df("CD", [0, 1])
display("df3", "df4", "pd.concat([df3, df4], axis='columns')")
# we could have equivalently specified axis=1; here we've used the more intuitive axis='columns'

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

In [47]:
# duplicate indices
# one important difference between np.concatenate and pd.concat is that pandas concatenation preserves indices, even if the result will have duplicate indices
x = make_df("AB", [0, 1])
y = make_df("AB", [2, 3])
y.index = x.index  # make indices match
display("x", "y", "pd.concat([x, y])")
# the repeated indices is the result. while this is valid within Dataframes the outcome is often undesirable. pd.concat() gives us a few ways to handle it

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


In [48]:
# treating repeated indices as an error
# to verify that the indices in the result of pd.concat() do not overlap you can include the verify_integrity flag.
# with this set to true the concatenation will raise an exception if there are duplicate indices,
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


In [49]:
# ignoring the index
# sometimes the index itself does not matter and you would prefer it to simply be ignored. this option can be specified using the ignore_index flag. with this set to true the concatenation will create a new integer index for the resulting dataframe
display("x", "y", "pd.concat([x, y], ignore_index=True)")

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [50]:
# adding multiindex keys
# another option is to use the keys option to specify a label for the data sources; the result will be a hierarchicallyj indexed series containing the data
display("x", "y", "pd.concat([x, y], keys=['x', 'y'])")
# we can use the tools discussed in hierarchical indexing to transform this multiply indexed dataframe into the representation we're interested in

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

Concatenation with joins

In [51]:
# in reality data from different sources might have different sets of column names. pd.concat() offers several options
# the following have some but not all columns in common
df5 = make_df("ABC", [1, 2])
df6 = make_df("BCD", [3, 4])
display("df5", "df6", "pd.concat([df5, df6])")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In [52]:
# the default behavior is to fill entries for which no dat is available with NA.
# to change this we can adjust the join parameter of the concat function. by default the join is a union of the input columns (join='outer'), but we can change this to an intersection of the columns using join='inner'
display("df5", "df6", 'pd.concat([df5, df6], join="inner")')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join="inner")
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

In [53]:
# another usefull pattern is to use the reindex method before concatenation for finer control over the columns dropped
pd.concat([df5, df6.reindex(df5.columns, axis="columns")])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


Merge and join operations

In [54]:
# relational algebra
# the behavior implimented in pd.merge is a subset of what is known as relational algebra, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases
# the categories of joins are one to one, many to many, and many to one
# all three are accessed via the pd.merge() interface; the type of join performed depends on the form of the input
# one to one joins which is similar to the column-wise concatenation seen erlier
df1 = pd.DataFrame(
    {
        "employee": ["Bob", "Jake", "Lisa", "Sue"],
        "group": ["Accounting", "Engineering", "Engineering", "HR"],
    }
)
df2 = pd.DataFrame(
    {"employee": ["Lisa", "Bob", "Jake", "Sue"], "hire_date": [2004, 2008, 2012, 2014]}
)
display("df1", "df2")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [60]:
# to combine this information into a single dataframe we can use the pd.merge() function
df3 = pd.merge(df1, df2)
df3
# the pd.merge function recognizes that each df has an employee column and auto joins using this column as a key into a new dataframe the order of each column is not necessarily maintained and the index is not particularly meaningful

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [58]:
# many to one joins in which one of the two key columns contains duplicate entries the resulting dataframe will preserve those duplicate entries as appropriate
df4 = pd.DataFrame(
    {
        "group": ["Accounting", "Engineering", "HR"],
        "supervisor": ["Carly", "Guido", "Steve"],
    }
)
display("df3", "df4", "pd.merge(df3, df4)")
# the resulting dataframe has an additional column with the supervisor where the information is repeated in one or more locations as required by the inputs

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [59]:
# many to many joins
# a bit confusting conceptually but well defined.
# if the key column in both the left and right array contains duplicates then the result is a many to many merge
# we have a dataframe showing one or more skills associated with a particular group
# by performing a many to many join we can recover the skills associated with any individual person
df5 = pd.DataFrame(
    {
        "group": ["Accounting", "Accounting", "Engineering", "Engineering", "HR", "HR"],
        "skills": [
            "math",
            "spreadsheets",
            "software",
            "math",
            "spreadsheets",
            "organization",
        ],
    }
)
display("df1", "df5", "pd.merge(df1, df5)")
# these three types of joins can be used with other pandas tools to implement a wide array of functionality in practice datasets are rarely as clean as the one we're working with here

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,software
3,Engineering,math
4,HR,spreadsheets


In [61]:
# specification of the merge key
# we've already seen the default behavior of pd.merge() it looks for one or more matching column names between the two inputs and uses this as the key. however often the column names will not match so pd.merge() provides a variety of options for handling this
display("df1", "df2", 'pd.merge(df1, df2, on="employee")')
# this option works only if both the left and right dataframes have the specified column name

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on="employee")
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

IMPORTANT!!!!!!!

In [62]:
# the left_on and right_on keywords
# at times you may wish to merge two datasets with different column names. such as one with name and one with employee in this case we can use the left_on and right_on keywords to specify the two column names
df3 = pd.DataFrame(
    {"name": ["Bob", "Jake", "Lisa", "Sue"], "salary": [70000, 80000, 120000, 90000]}
)
display("df1", "df3", 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

In [63]:
# the result has a redundant column that we can drop if desired for example by using the drop() method of DataFrames
pd.merge(df1, df3, left_on="employee", right_on="name").drop("name", axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [64]:
# the left_index and right_index keywords
# sometimes rather than merging on a column you would instead like to merge on an index
df1a = df1.set_index("employee")
df2a = df2.set_index("employee")
display("df1a", "df2a")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [65]:
# you can use the index as the key for merging by specifying the left_index and/or right_index flags in pd.merge()
display("df1a", "df2a", "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [66]:
# for convenience Dataframes.join() method exists which performs an index based merge without extra keywords
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [68]:
# if you'd like to mix indices and columns you can combine left_index with right_on or left_on with right_index to get the desired behavior
display("df1a", "df3", 'pd.merge(df1a, df3, left_index=True, right_on="name")')
# all of these options also work with multiple indices and/or multiple columns; the interface for this behavior is very intuitive see merge join and concatenate in documentation

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on="name")
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

In [69]:
# specifiying set arithmetic for joins
# the type of set arithmetic used in the join is important to consideer. this comes up when a value appears in one key column but not the other
df6 = pd.DataFrame(
    {"name": ["Peter", "Paul", "Mary"], "food": ["fish", "beans", "bread"]},
    columns=["name", "food"],
)
df7 = pd.DataFrame(
    {"name": ["Mary", "Joseph"], "drink": ["wine", "beer"]}, columns=["name", "drink"]
)
display("df6", "df7", "pd.merge(df6, df7)")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

In [70]:
# here we have merged two datasets that have only a single name entry in common: Mary. by default the result contains the intersection of the two sets of inputs this is what is known as an inner join. we can specify this explicitly using the how keyword which defaults to inner
pd.merge(df6, df7, how="inner")

,name,food,drink
0,Mary,bread,wine


In [71]:
# other options for the how keyword are 'outer', 'left' and 'right' an outer join returns a join over the union of the input columns and fills in all missing values with NAs
display("df6", "df7", 'pd.merge(df6, df7, how="outer")')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how="outer")
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

In [72]:
# the left join and right join return joins over the left entries and right entries respectively
display("df6", "df7", 'pd.merge(df6, df7, how="left")')
# the output rows now correspond to the entries in the left input using how="right" works in a similar manner

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how="left")
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [73]:
# overlapping column names: the suffixes keyword
# finally you may end up in a case where your two input dataframes have conflicting column names
df8 = pd.DataFrame({"name": ["Bob", "Jake", "Lisa", "Sue"], "rank": [1, 2, 3, 4]})
df9 = pd.DataFrame({"name": ["Bob", "Jake", "Lisa", "Sue"], "rank": [3, 1, 4, 2]})
display("df8", "df9", 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

In [74]:
# because the output would have two conflicting column names the merge function automatically appends the suffixes _x and _y to make the output columns unique it is possible to specify a custom suffix using the suffixes keyword
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
# these suffixes work in any of the possible join patterns and also work if there are multiple overlapping columns

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Example: US States Data

In [75]:
populations = pd.read_csv("Data/USstates-master/state-population.csv")
areas = pd.read_csv("Data/USstates-master/state-areas.csv")
abbrevs = pd.read_csv("Data/USstates-master/state-abbrevs.csv")

display("populations.head()", "areas.head()", "abbrevs.head()")

populations.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

In [76]:
# given this information say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density. We clearly have the data here to find this result but we'll have to combine the datasets to get it
# We'll start with a many-to-one merge that will give us the full state names within the population DataFrame. We want to merge based on the state/region column of pop and the abbreviation column of abbrevs. We'll use how='outer' to make sure no data is thrown away due to mismatched labels
merged = pd.merge(
    populations, abbrevs, how="outer", left_on="state/region", right_on="abbreviation"
)
merged = merged.drop("abbreviation", axis=1)  # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [77]:
# we can check for mismatches by looking for rows with nulls
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [78]:
# some of the population values are null; let's figure out which these are!
merged[merged["population"].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [79]:
# all the null population values are from puerto rico prior to the year 2000 this is likely due to the data not being available from the original source
# we also see that some of the new state entries are also null which means that there was no corresponding entry in the abbrevs key! let's figure out which regions lack this match
merged.loc[merged["state"].isnull(), "state/region"].unique()

array(['PR', 'USA'], dtype=object)

In [81]:
# we can see that all the null values are from puerto rico and usa we can fix this by filling in appropriate entries
merged.loc[merged["state/region"] == "PR", "state"] = "Puerto Rico"
merged.loc[merged["state/region"] == "USA", "state"] = "United States"
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [82]:
# this leaves no more null values in the state column
# we can merge the result with the area data using a similar procedure. we will want to join on the state column in both
final = pd.merge(merged, areas, on="state", how="left")
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [83]:
# let's check to see if there are any mismatches
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [84]:
# there are nulls in the area column; we can take a look to see which regions were ignored here
final["state"][final["area (sq. mi)"].isnull()].unique()

array(['United States'], dtype=object)

In [85]:
# we see that our areas df does not contain the area of the us as a whole we could insert the appropriate value (summing all the state areas) but in this case we'll just drop the null values because the population density of the entire us is not relevant to our current discussion
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [86]:
# we have all the data we need we first select the portion of the data corresponding with the year 2010 and the total population we can use the query() funtion to do this quickly
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [87]:
# now let'a compute the population density and display it in order. we'll start by reindexing our data on the state and then compute the result
data2010.set_index("state", inplace=True)
density = data2010["population"] / data2010["area (sq. mi)"]
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [88]:
# this result is a ranking of us states plus dc and puerto rico in order of their 2010 population density in residents per square mile we can see that by far the densest region in this dataset is dc at almost 9000 residents per square mile approximately 9 times the density of the runner up new jersey
# we can also check the end of the list
density.tail()
# this type of data merging is a common task when trying to answer questions using real world data sources

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64